In [2]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import matplotlib.pyplot as plt
import datetime
import matplotlib.dates as mdates
%matplotlib inline 
!pip3 install mpld3
import mpld3
mpld3.enable_notebook()

from scipy.integrate import odeint
!pip3 install lmfit
import lmfit
from lmfit.lineshapes import gaussian, lorentzian, exponential

import warnings
warnings.filterwarnings('ignore')
# JSM added
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# !! if you get a timeout-error, just click on the link and download the data manually !!

# read the data
beds = pd.read_csv("https://raw.githubusercontent.com/hf2000510/infectious_disease_modelling/master/data/beds.csv", header=0)
agegroups = pd.read_csv("https://raw.githubusercontent.com/hf2000510/infectious_disease_modelling/master/data/agegroups.csv")
probabilities = pd.read_csv("https://raw.githubusercontent.com/hf2000510/infectious_disease_modelling/master/data/probabilities.csv")
covid_data = pd.read_csv("https://tinyurl.com/t59cgxn", parse_dates=["Date"], skiprows=[1])
covid_data["Location"] = covid_data["Country/Region"]

# create some dicts for fast lookup
# 1. beds
beds_lookup = dict(zip(beds["Country"], beds["ICU_Beds"]))
# 2. agegroups
agegroup_lookup = dict(zip(agegroups['Location'], agegroups[['0_9', '10_19', '20_29', '30_39', '40_49', '50_59', '60_69', '70_79', '80_89', '90_100']].values))

# store the probabilities collected
prob_I_to_C_1 = list(probabilities.prob_I_to_ICU_1.values)
prob_I_to_C_2 = list(probabilities.prob_I_to_ICU_2.values)
prob_C_to_Death_1 = list(probabilities.prob_ICU_to_Death_1.values)
prob_C_to_Death_2 = list(probabilities.prob_ICU_to_Death_2.values)

In [5]:
agegroup_lookup

{'World': array([1.342381e+09, 1.253463e+09, 1.192080e+09, 1.150350e+09,
        9.731550e+08, 8.336220e+08, 5.917860e+08, 3.124590e+08,
        1.241160e+08, 2.081400e+07]),
 'Sustainable Development Goal (SDG) regions': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'Sub-Saharan Africa': array([3.24399e+08, 2.53430e+08, 1.85189e+08, 1.33417e+08, 8.92920e+07,
        5.64640e+07, 3.33850e+07, 1.51200e+07, 3.47700e+06, 1.91000e+05]),
 'Eastern Africa': array([1.30038e+08, 1.05675e+08, 7.81560e+07, 5.41080e+07, 3.52710e+07,
        2.17140e+07, 1.28490e+07, 5.97800e+06, 1.51300e+06, 1.02000e+05]),
 'Burundi': array([3.873e+06, 2.725e+06, 2.030e+06, 1.539e+06, 7.480e+05, 4.920e+05,
        3.390e+05, 1.120e+05, 3.000e+04, 2.000e+03]),
 'Comoros': array([238000., 192000., 151000., 118000.,  77000.,  51000.,  30000.,
         12000.,   3000.,      0.]),
 'Djibouti': array([197000., 180000., 177000., 162000., 121000.,  78000.,  44000.,
         22000.,   6000.,      0.]),
 'Eri

In [7]:
len(agegroup_lookup.keys())

241

In [9]:
prob_I_to_C_1

[0.001, 0.003, 0.009000000000000001, 0.025, 0.03, 0.05, 0.07, 0.1, 0.14, 0.16]

In [10]:
prob_C_to_Death_1

[0.0, 0.0, 0.05, 0.06, 0.1, 0.26, 0.57, 0.86, 0.95, 0.98]